# Compute the dynamics of the network homophily

In [1]:
import sys
import importlib

sys.path.insert(0, '../local/bin')

import edgelist_creation
from edgelist_creation import *

In [2]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import LineString

In [3]:
importlib.reload(edgelist_creation)

<module 'edgelist_creation' from '/home/leonardo/projects/USA_Congress_SigNet/notebooks/../local/bin/edgelist_creation.py'>

In [4]:
members = pd.read_csv('../dataset/HSall_members.csv')			# Congressmen from G. Washington to D. Trump
party_codes = pd.read_csv('../dataset/HSall_parties.csv')		# Parties to map party_code to party_name

In [5]:
members_info = create_members_df(members, party_codes)
members_info.head(10)

,icpsr,congress,party_code,bioname,state_abbrev,party_name
0,1,[82],[200],"AANDAHL, Fred George",ND,[Republican]
1,2,"[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...",[100],"ABBITT, Watkins Moorman",VA,[Democrat]
2,3,"[28, 29, 30]",[29],"ABBOTT, Amos",MA,[Whig]
3,4,"[50, 51, 52, 53, 54]",[100],"ABBOTT, Jo (Joseph)",TX,[Democrat]
4,5,"[15, 16, 17, 18]","[7777, 13]","ABBOT, Joel",GA,"[Crawford Republican, Democrat-Republican]"
5,6,"[40, 41]",[200],"ABBOTT, Joseph Carter",NC,[Republican]
6,7,[44],[100],"ABBOTT, Josiah Gardner",MA,[Democrat]
7,8,[35],[200],"ABBOTT, Nehemiah",ME,[Republican]
8,9,[83],[200],"ABEL, Hazel Hempel",NE,[Republican]
9,10,"[32, 33]",[29],"ABERCROMBIE, James",AL,[Whig]


In [24]:
member_party_dict = members_info.set_index('icpsr')['party_name'].to_dict()		# member_id -> party_name
member_congress_dict = members_info.set_index('icpsr')['congress'].to_dict()	# member_id -> congress
member_state_dict = members_info.set_index('icpsr')['state_abbrev'].to_dict()	# member_id -> state_abbreviation

### Edgelist from congress

In [20]:
congress_77 = pd.read_csv('../dataset/download_votes_merged/congress_077.csv')
congress_77.head(3)

,bill_number,id,description,chamber,date,vote_desc,vote_result,congress,rollnumber,Peltzman,Clausen,name,party_short_name,icpsr,vote
0,HR3,RH0770071,"TO STRIKE THE ENACTING CLAUSE OF H.R. 3, A BIL...",House,1941-11-18,NaN,NaN,77,71,Domestic Social Policy,Social Welfare,"ROOSEVELT, Franklin Delano",Democrat,99899,Abs
1,HR3,RH0770071,"TO STRIKE THE ENACTING CLAUSE OF H.R. 3, A BIL...",House,1941-11-18,NaN,NaN,77,71,Domestic Social Policy,Social Welfare,"BOYKIN, Frank William",Democrat,937,Abs
2,HR3,RH0770071,"TO STRIKE THE ENACTING CLAUSE OF H.R. 3, A BIL...",House,1941-11-18,NaN,NaN,77,71,Domestic Social Policy,Social Welfare,"GRANT, George McInvale",Democrat,3754,Abs


In [17]:
congress_77['state'] = congress_77['icpsr'].map(member_state_dict)
congress_77.head(3)

,bill_number,id,description,chamber,date,vote_desc,vote_result,congress,rollnumber,Peltzman,Clausen,name,party_short_name,icpsr,vote,state
0,HR3,RH0770071,"TO STRIKE THE ENACTING CLAUSE OF H.R. 3, A BIL...",House,1941-11-18,NaN,NaN,77,71,Domestic Social Policy,Social Welfare,"ROOSEVELT, Franklin Delano",Democrat,99899,Abs,USA
1,HR3,RH0770071,"TO STRIKE THE ENACTING CLAUSE OF H.R. 3, A BIL...",House,1941-11-18,NaN,NaN,77,71,Domestic Social Policy,Social Welfare,"BOYKIN, Frank William",Democrat,937,Abs,AL
2,HR3,RH0770071,"TO STRIKE THE ENACTING CLAUSE OF H.R. 3, A BIL...",House,1941-11-18,NaN,NaN,77,71,Domestic Social Policy,Social Welfare,"GRANT, George McInvale",Democrat,3754,Abs,AL


In [44]:
edgelist = prova_state_edgelist_from_congress(congress_77, member_state_dict)

100%|██████████| 152/152 [00:34<00:00,  4.43it/s]
